In [ ]:
import pathlib as pl
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon
import flopy

In [ ]:
# load mfusg surface water model grid
fpth = pl.Path("./sorab/_abdul_build/abdul-mc.swf.gsf")
mg = flopy.discretization.UnstructuredGrid.from_gridspec(fpth)
mg.plot()

In [ ]:
fpth = pl.Path("./sorab/_abdul_build/abdul-mc.dis")
mfusg = flopy.mfusg.MfUsg()
dis = flopy.mfusg.MfUsgDisU.load(fpth, model=mfusg)
land_surface = dis.top[0].array
dis.bot.array.shape

In [ ]:
# flip iverts to be clockwise
iverts = mg.iverts
for i in range(mg.nnodes):
    iv = iverts[i]
    ivr = iv[::-1]
    iverts[i] = ivr

In [ ]:
# calculate area for each cell
area = np.array([Polygon(mg.get_cell_vertices(icpl)).area for icpl in range(mg.nnodes)])

In [ ]:
# set up swf model discretization based on mfusg version
disv_gridprops = flopy.utils.cvfdutil.get_disv_gridprops(mg.verts, iverts)
disv_gridprops["nodes"] = disv_gridprops.pop("ncpl")
disv_gridprops["bottom"] = land_surface
disv_gridprops.keys()

In [ ]:
# set up gwf model discretization based on mfusg version
top = land_surface
botm = [b.array for b in dis.bot]
nlay = 15
ncpl = mg.nnodes
disv_gwf_gridprops = flopy.utils.cvfdutil.get_disv_gridprops(mg.verts, iverts)
disv_gwf_gridprops["nlay"] = 15
disv_gwf_gridprops["top"] = top
disv_gwf_gridprops["botm"] = botm

In [ ]:
exe_name = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/mf6"
sim_ws = pl.Path(f"./mf6_abdul")

# rainfall applied to topnodes for fifty minutes
perlen = 2 * [50. * 60] # 50 mins converted to seconds
rainfall = [2 / 100. / 60 / 60, 0.] # 2 cm per hour converted to m/s; second stress period is 0.
mannings_channel = 0.03 # m/s^1/3
mannings_slopes = 0.3 # m/s^1/3
initial_water_depth = 1.e-4 # m
timestep_minimum = 0.5 # s
timestep_maximum = 100 # s
zdg_nodes = [2310, 2395, 2396, 2466, 2467, 2518, 2519, 2567, 2569, 2610, 2612, 2639, 2641, 2650]

kx = 1.e-5
kz = 1.e-5
ss = 1.2e-7
sy = 0.34

In [ ]:
iz = np.array(zdg_nodes) - 1
ibd = np.zeros(mg.nnodes, dtype=int)
ibd[iz] = 1
fig, ax = plt.subplots(figsize=(8, 4), constrained_layout=True)
ax.set_aspect(1)
ax.set_xlabel(r'x')
ax.set_ylabel(r'y')
title = ax.set_title(f"ZDG Nodes")
pmv = flopy.plot.PlotMapView(modelgrid=mg, ax=ax)
pmv.plot_array(ibd)

In [ ]:
def add_swf_model(sim):
    # surface water model
    swfname = f"swf_model"
    swf = flopy.mf6.ModflowSwf(sim, modelname=swfname, save_flows=True)

    disv = flopy.mf6.ModflowSwfdisv2D(
        swf,
        **disv_gridprops
    )

    dfw = flopy.mf6.ModflowSwfdfw(
        swf,
        print_flows=False,
        save_flows=True,
        save_velocity=True,
        manningsn=mannings_slopes,
        idcxs=None,
    )

    sto = flopy.mf6.ModflowSwfsto(
        swf,
        save_flows=True,
        steady_state={0: False, 1: False},
        transient={0: True, 1: True},
    )

    ic = flopy.mf6.ModflowSwfic(
        swf,
        strt=land_surface + initial_water_depth,
    )

    # output control
    oc = flopy.mf6.ModflowSwfoc(
        swf,
        budget_filerecord=f"{swfname}.bud",
        stage_filerecord=f"{swfname}.stage",
        saverecord=[
            ("STAGE", "ALL"),
            ("BUDGET", "ALL"),
        ],
        printrecord=[
            ("BUDGET", "ALL"),
        ],
    )

    # flw
    spd = [((icpl,), area[icpl] * rainfall[0]) for icpl in range(mg.nnodes)]
    flw = flopy.mf6.ModflowSwfflw(
        swf,
        maxbound=len(spd),
        print_input=True,
        print_flows=True,
        stress_period_data={0: spd, 1: []},
    )

    fname = f"{swfname}.outflow.csv"
    zdg_obs = {
        fname: [
            (f"OUTFLOW{izdg}", "ZDG", (izdg - 1,)) for izdg in zdg_nodes
        ],
        "digits": 10,
    }

    idcxs = -1
    zdg_slope = 0.001
    width = 1
    spd = [
        ((izdg - 1,), idcxs, width, zdg_slope, mannings_channel) for izdg in zdg_nodes
    ]
    zdg = flopy.mf6.ModflowSwfzdg(
        swf,
        observations=zdg_obs,
        print_input=True,
        maxbound=len(spd),
        
        stress_period_data=spd,
    )

    # obs_data = {
    #     f"{swfname}.obs.csv": [
    #         ("STAGE", "STAGE", (iobs_row, iobs_col)),
    #         ("STAGE0", "STAGE", (0, iobs_col)),
    #         ("FLOW", "FLOW-JA-FACE", (iobs_row, 7), (iobs_row + 1, iobs_col)),
    #         ("FLOW0", "FLOW-JA-FACE", (0, iobs_col), (1, iobs_col)),
    #     ],
    # }
    # obs_package = flopy.mf6.ModflowUtlobs(
    #     swf,
    #     filename=f"{swfname}.obs",
    #     digits=10,
    #     print_input=True,
    #     continuous=obs_data,
    # )

    return swf

In [ ]:
def add_gwf_model(sim):

    # create gwf model
    name = "gwf_model"
    gwf = flopy.mf6.ModflowGwf(
        sim,
        modelname=name,
        save_flows=True,
        newtonoptions="UNDER_RELAXATION",
    )

    disv = flopy.mf6.ModflowGwfdisv(
        gwf,
        **disv_gwf_gridprops
    )

    # initial conditions
    ic = flopy.mf6.ModflowGwfic(
        gwf, 
        # strt=2.78, # If the water table is set low, then all rainfall goes into subsurface
        strt = nlay * [land_surface],
    )

    # node property flow
    npf = flopy.mf6.ModflowGwfnpf(
        gwf,
        save_flows=False,
        icelltype=1,
        k=kx,
        k33=kz,
    )

    sto = flopy.mf6.ModflowGwfsto(
        gwf, 
        sy=sy, 
        ss=ss, 
        iconvert=1,
        steady_state={0: False, 1: False},
        transient={0: True, 1: True},        
    )

    # output control
    oc = flopy.mf6.ModflowGwfoc(
        gwf,
        budget_filerecord=f"{name}.bud",
        head_filerecord=f"{name}.hds",
        headprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
        saverecord=[
            ("HEAD", "ALL"), 
            ("BUDGET", "ALL")
        ],
        printrecord=[
            # ("HEAD", "LAST"), 
            ("BUDGET", "ALL")
        ],
    )
    return gwf

In [ ]:
def add_exchange(sim):
    swfgwf_data = []
    leakance = kz / 0.5
    for icpl in range(ncpl):
        cfact = area[icpl]
        t = dis.top[0].array[icpl]
        b = dis.bot[0].array[icpl]
        dz = t - b
        leakance = kz / dz
        exg = ((icpl,), (0, icpl), leakance, cfact)
        swfgwf_data.append(exg)
    swfgwf = flopy.mf6.ModflowSwfgwf(
        sim,
        print_input=False,
        print_flows=False,
        exgtype="SWF6-GWF6",
        nexg=len(swfgwf_data),
        exgmnamea="swf_model",
        exgmnameb="gwf_model",
        exchangedata=swfgwf_data,
    )
    return swfgwf

In [ ]:
# make modflow model
# build MODFLOW 6 files
name = "mysim"
sim = flopy.mf6.MFSimulation(
    sim_name=name,
    version="mf6",
    exe_name=exe_name,
    sim_ws=sim_ws,
)

# create tdis package
nper = len(perlen)
tdis = flopy.mf6.ModflowTdis(
    sim,
    time_units="SECONDS",
    nper=nper,
    perioddata=[(perlen[i], 1, 1.0) for i in range(nper)],
)

ats_filerecord = name + ".ats"
atsperiod = [
    (0, timestep_minimum, timestep_minimum, timestep_maximum, 1.5, 4),
    (1, timestep_minimum, timestep_minimum, timestep_maximum, 1.5, 4),
]
tdis.ats.initialize(
    maxats=len(atsperiod),
    perioddata=atsperiod,
    filename=ats_filerecord,
)

# add models and exchanges
swf = add_swf_model(sim)
gwf = add_gwf_model(sim)
exg = add_exchange(sim)

nouter, ninner = 100, 25
hclose = 1.e-4
relax = 0.  #0.97
ims = flopy.mf6.ModflowIms(
    sim,
    print_option="SUMMARY",
    outer_dvclose=hclose,
    outer_maximum=nouter,
    under_relaxation="DBD",
    under_relaxation_theta=0.9,
    under_relaxation_kappa=0.0001,
    under_relaxation_gamma=0.0,
    inner_maximum=ninner,
    inner_dvclose=hclose,
    linear_acceleration="BICGSTAB",
    scaling_method="NONE",
    reordering_method="NONE",
    relaxation_factor=relax,
    preconditioner_levels=3,
    backtracking_number=5,
    backtracking_tolerance=1.0,
    backtracking_reduction_factor=0.3,
    backtracking_residual_limit=100.0,
    filename=f"{name}.ims",
)
model_list = [swf.name, gwf.name]
sim.register_ims_package(ims, model_list)

# # write the model files
# sim.write_simulation()

# # run the model
# sim.run_simulation()

In [ ]:
fpth = sim_ws / f"{sim.swf[0].name}.outflow.csv"
obsvals = np.genfromtxt(fpth, names=True, delimiter=",")
outflow = []
for row in obsvals:
    row_sum = 0.
    for field in obsvals.dtype.fields:
        if "outflow" in field.lower():
            value = row[field] # value is in m^3/s
            value = value * 1000. # convert from m^3/s to L/s
            value = value * 60 # convert L/s to L/min
            row_sum += value
    outflow.append(-row_sum)

plt.plot(obsvals["time"] / 60., outflow)
plt.xlabel("time, in minutes")
plt.ylabel("")

In [ ]:
fpth = sim_ws / f"{sim.swf[0].name}.stage"
stage_obj = flopy.utils.HeadFile(fpth, text="STAGE")
stage = stage_obj.get_data().flatten()


In [ ]:
# stage
fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot(1, 1, 1)
ax.set_aspect(1.)
swf = sim.swf[0]
pmv = flopy.plot.PlotMapView(modelgrid=mg)
qm = pmv.plot_array(stage)


In [ ]:
# stage
fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot(1, 1, 1)
ax.set_aspect(1.)
swf = sim.swf[0]
pmv = flopy.plot.PlotMapView(modelgrid=mg)
qm = pmv.plot_array(stage - land_surface)

In [ ]:
times = swf.output.stage().get_times()
fig, ax = plt.subplots(figsize=(8, 4), constrained_layout=True)
ax.set_aspect(1)
ax.set_xlabel(r'x')
ax.set_ylabel(r'y')
title = ax.set_title(f"Time = {times[0]} seconds")

pmv = flopy.plot.PlotMapView(modelgrid=mg, ax=ax)
# pmv.plot_grid(lw=0.5, color="0.5")
stage_alldata = stage_obj.get_alldata()
depth_alldata = [(s - land_surface).flatten() for s in stage_alldata]
depth_alldata = np.array(depth_alldata)
ca_dict = {
    "vmin": 0,
    "vmax": depth_alldata.max(),
}
c = depth_alldata[0]
cont = pmv.plot_array(c, **ca_dict)
clb = fig.colorbar(
    cont, 
    shrink=0.5, 
)

def animate(i):
    c = depth_alldata[i]
    cont.set_array(c)
    title = ax.set_title(f"Time = {times[i]:.2f} seconds")
    return cont

import matplotlib.animation
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=stage_alldata.shape[0])
plt.close()

from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
from flopy.export.vtk import Vtk

vtk = Vtk(model=gwf, binary=False, vertical_exageration=50, smooth=False)
vtk.add_model(gwf)
# vtk.add_pathline_points(pl)

In [ ]:
pvgrid = vtk.to_pyvista()
pvgrid

In [ ]:
import pyvista as pv

axes = pv.Axes(show_actor=True, actor_scale=2.0, line_width=5)
pvgrid.rotate_z(160, point=axes.origin, inplace=True)

pv.set_plot_theme("document")
# pv.set_jupyter_backend("static")
pv.set_jupyter_backend('trame')

# create the plot and add the grid and pathline meshes
p = pv.Plotter()
p.add_mesh(pvgrid, opacity=0.05)

p.camera.zoom(2.4)
p.show()